In [1]:
import warnings
warnings.filterwarnings("ignore")

from transformers import pipeline
import pandas as pd

In [2]:
text = """
    Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expecto to hear from you soon. Sincerely, Bumblebee
    """

In [3]:
# sentiment analysis
classifier = pipeline("text-classification")
outputs = classifier(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,label,score
0,NEGATIVE,0.970257


In [4]:
# ner: named entity recognition
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,entity_group,score,word,start,end
0,ORG,0.884254,Amazon,10,16
1,MISC,0.991045,Optimus Prime,41,54
2,LOC,0.999749,Germany,95,102
3,MISC,0.560742,Mega,213,217
4,PER,0.587686,##tron,217,221
5,ORG,0.685692,Decept,258,264
6,MISC,0.488548,##icons,264,269
7,MISC,0.778458,Megatron,355,363
8,MISC,0.987869,Optimus Prime,372,385
9,PER,0.744715,Bumblebee,508,517


In [5]:
# question answering
reader = pipeline("question-answering")
question = "What does the customer want?"
outputs = reader(question=question, context=text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.623638,340,363,an exchange of Megatron


In [6]:
# summarization
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=45, clean_up_tokenization_spaces=True)
outputs

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Your min_length=56 must be inferior than your max_length=45.


[{'summary_text': ' Bumblebee demands an exchange of Megatron for the Optimus Prime figure he bought from Amazon. The Decepticons are a lifelong enemy of the Decepticon, and the Autobot is a lifelong foe of'}]

In [7]:
# translation
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
outputs

[{'translation_text': 'Liebe Amazon, letzte Woche bestellte ich eine Optimus Prime Action Figur von Ihrem Online-Shop in Deutschland. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich eine Action Figur von Megatron geschickt worden war statt! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Prime Figur bestellte ich. Anbei sind Kopien meiner Aufzeichnungen über diesen Kauf. Ich erwarte, von Ihnen bald zu hören. Aufrichtig, Bumblebee'}]

In [8]:
# text generation
generator = pipeline("text-generation")
response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response
outputs = generator(prompt, max_length=200)
outputs

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': '\n    Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. Enclosed are copies of my records concerning this purchase. I expecto to hear from you soon. Sincerely, Bumblebee\n    \n\nCustomer service response:\nDear Bumblebee, I am sorry to hear that your order was mixed up. The item was in perfect condition and in good working order. I am currently trying to contact you to resolve this ASAP. Can you resolve this asap? Thank you. Sorry for the inconvenience.\n\nPlease contact me immediately if you have any questions.\n\nI cannot accept return and'}]